In [1]:
import datetime
import random
from os import listdir

In [2]:
DATA_DIR = "data"
blocklist = (
"91481.html", # k.o.
"9550.html", # UV
"91478.html", # dziecino
"29140.html", # kierowniku
"90382.html", # chłopaczku
"40977.html", # człowiecze
"29141.html", # kierowniczko
"61928.html", # koleżko
"31183.html", # myszko
"59776.html", # kiciuniu
"29618.html", # kotku
"65072.html", # chłopie
"41029.html", # człeku
"105221.html", # kochasiu
"92571.html", # bratku
"15734.html", # brachu
"37342.html", # rybko
"41028.html", # człecze
"45533.html", # kochani
"87117.html", # chłopaku
"42963.html", # przyjacielu
"92572.html", # braciszku
"28077.html", # królowo
"59772.html", # kiciu
"40969.html", # człowieku
"44341.html", # gościu
"91476.html", # kolego
"40942.html", # mała
"40954.html", # maleńka
"29142.html", # szefowo
"15685.html", # córko
"15684.html", # synu
"29143.html", # szefie
"47286.html", # kochanku
"105930.html", # mordo

"112658.html", # graty
"113841.html", # gratki

"37353.html", # dwa II
"37384.html", # trzy II
"37383.html", # cztery II
"37404.html", # pięć II
"37406.html", # sześć II
"87728.html", # celująco II
"59113.html", # dostatecznie I

"9327.html", # kto II
"9320.html", # ktokolwiek I
"53640.html", # ja
"52918.html", # ty I
"53688.html", # on I
"53663.html", # my
"53863.html", # wy
"26512.html", # nikt
"91668.html", # ten II    
"61655.html", # co II
"41031.html", # to I
"21502.html", # oto I
"29385.html", # ot I

"9326.html", # cokolwiek I
"83880.html", # cośkolwiek I
"96872.html", # tamten II
"6110.html", # nic I
"6111.html", # nic II
"89526.html", # wszystko
"90075.html", # taki II
    
"107705.html", # pele-mele
"99234.html", # toto
"60211.html", # liposuction
"90232.html", # figle-migle
"71698.html", # burkini
"96431.html", # korpo
"68907.html", # kabrio
"91596.html", # homoniewiadomo
"83503.html", # rooming-in
"79130.html", # pierdu, pierdu
"20904.html", # albo-albo
"110375.html", # fiku-miku II
"21600.html", # albo, albo
"77067.html", # szacher-macher
"109689.html", # Dzopa
"109687.html", # Dropa
"109365.html", # furda
"107647.html", # wódżitsu
"75741.html", # trele-morele
"117123.html", # wice
"59210.html", # mung
"100085.html", # uni
"108653.html", # fi I
"30665.html", # teraz II
"96428.html", # ideolo I

"38177.html", # jedno
"35444.html", # niejedno
    
"11063.html", # rad I
"106306.html", # ścieki
"89331.html", # wypominki

"47979.html", # śmigus-dyngus I
"88723.html", # esy-floresy
"58651.html", # klub-kawiarnia
"81977.html", # cmok-nonsens

"48119.html", # rzeczownik zbiorowy
)
filenames = [x for x in listdir(DATA_DIR) if x.find("html") > 0 and x not in blocklist]
len(filenames)

84308

In [3]:
def is_noun(s):
    return s.find("<em>rzeczownik</em>") >= 0

def extract_tag_value(s, tag, offset=0, tag_short=None):
    if tag_short is None:
        tag_short = tag
    a = s.find(f"<{tag}", offset)
    a = s.find(">", a)
    b = s.find(f"</{tag_short}>", a)
    return s[a + 1 : b].strip(), b + len(tag_short) + 3

def extract_tag_values(s, tag, offset=0):
    res = []
    
    while s.find(f"<{tag}", offset) >= 0:
        word, offset = extract_tag_value(s, tag, offset)
        res.append(word)
    return res

def parse_inflection(s):
    res = []
    offset = 0

    modifiers = ["char", "neut", "depr", "ndepr"]
    
    while s.find("<td", offset) >= 0:
        td, offset = extract_tag_value(s, "td", offset)
        spans = extract_tag_values(td, 'span')
        res.append(spans)

    for i, x in enumerate(res):
        if len(x) == 1 and x[0] in modifiers and i > 0:
            for j, _ in enumerate(res[i - 1]):
                res[i - 1][j] += f" ({x[0]})"

    res = [x for x in res if not (len(x) == 1 and x[0] in modifiers)]

    l = len(res)
    while l > 2 and len(res[l - 1]) == 0:
        l -= 1
    return res[:l]

def parse_inflection_table(s):
    res = []
    offset = 0

    while s.find('<th', offset) >= 0:
        th, offset = extract_tag_value(s, 'th', offset)
        if th in ['', '<em>liczba pojedyncza</em>', '<em>liczba mnoga</em>']:
            continue
        
        case_name, _ = extract_tag_value(th, 'span')

        next_header_pos = s.find('<th', offset)
        if next_header_pos < 0:
            next_header_pos = len(s)
        
        inflection = parse_inflection(s[offset:next_header_pos])
        if len(case_name) > 0 or len(inflection) > 0:
            res.append([case_name] + inflection)
    return res

def parse_category(s):
    return [x.strip() for x in s.split('<i class="fas fa-long-arrow-alt-right"></i>')]

def parse_categories(s):
    res = []
    offset = 0

    while s.find("<p", offset) >= 0:
        word, offset = extract_tag_value(s, "p", offset)
        res.append(parse_category(word))
    return res

def parse_part_of_speech(s):
    word, offset = extract_tag_value(s, "em", 0)
    return word

def parse_genders(s):
    res = []
    offset = 0

    while s.find("em", offset) >= 0:
        word, offset = extract_tag_value(s, "em", offset)
        if len(word) > 0:
            res.append(word)
    return res


In [4]:
class Word:
    def __init__(self, word_id, entry, part_of_speech, genders, subentry, categorization, inflection):
        self.word_id = word_id
        self.entry = entry
        self.part_of_speech = part_of_speech
        self.genders = genders
        self.subentry = subentry
        self.categorization = categorization
        self.inflection = inflection

    def __str__(self):
     return "word_id: " + str(self.word_id) + ",\n" + \
        "entry: " + str(self.entry) + ",\n" + \
        "part_of_speech: " + str(self.part_of_speech) + ",\n" + \
        "genders: " + str(self.genders) + ",\n" + \
        "subentry: " + str(self.subentry) + ",\n" + \
        "categorization: " + str(self.categorization) + ",\n" + \
        "inflection: " + str(self.inflection) + "\n"

In [5]:
start = datetime.datetime.now()
print(f"START {start.time()}")

words = []

for filename in filenames: # [x for x in filenames if "85274" in x]:
    with open(f"{DATA_DIR}/{filename}", "r") as file:
        word_id = int(filename.split(".")[0])
        file_contents = file.read()
        file_contents = file_contents.replace("depr</td>", "depr</span></td>")
        if not is_noun(file_contents):
            continue

        h1, offset = extract_tag_value(file_contents, "h1")
        h2, categories = None, None

        while True:
            current_header_pos = file_contents.find("<h", offset)
            if current_header_pos < 0:
                break

            next_header_pos = file_contents.find("<h", current_header_pos + 1)
            if next_header_pos < 0:
                next_header_pos = len(file_contents)
            
            if file_contents[current_header_pos + 2] == '2': # h2
                if h2 is not None:
                    words.append(Word(word_id, h1, part_of_speech, genders, h2, categories, inflection_table))
                    categories = None        
                h2, offset = extract_tag_value(file_contents, "h2", offset)
            else: # h3
                h3, offset = extract_tag_value(file_contents, "h3", offset)
                if h3 == 'Odmiana':
                    p, offset = extract_tag_value(file_contents, "p", offset)
                    part_of_speech = parse_part_of_speech(p)

                    if file_contents[offset:next_header_pos].find("<p") < 0:
                        h2 = None
                        continue
                    
                    p, offset = extract_tag_value(file_contents, "p", offset)
                    genders = parse_genders(p)
    
                    inflection_table = parse_inflection_table(file_contents[offset:next_header_pos])
                    offset = next_header_pos
                else: # Kwalifikacja tematyczna
                    categories = parse_categories(file_contents[offset:next_header_pos])
                    offset = next_header_pos

        words.append(Word(word_id, h1, part_of_speech, genders, h2, categories, inflection_table))

end = datetime.datetime.now()
print(f"END   {end.time()}")
delta_seconds = (end - start).total_seconds()
print(f"elapsed:   {datetime.timedelta(seconds=int(delta_seconds))}")

START 23:25:04.659823
END   23:25:34.756648
elapsed:   0:00:30


In [6]:
print(len(words))

{
    k: len([x for x in words if ((x.categorization is None) if k == 0 else (x.categorization is not None and len(x.categorization) == k))])
    for k in set([len(x.categorization) for x in words if x.categorization is not None]) | {0}
}

60384


{0: 213, 1: 41447, 2: 16853, 3: 1825, 4: 46}

In [7]:
for word in random.choices(words, k=2):
    print(word)

word_id: 11455,
entry: postęp,
part_of_speech: rzeczownik,
genders: ['m3'],
subentry: 1. proces,
categorization: [['KATEGORIE FIZYCZNE', 'Cechy i właściwości materii', 'jakość i intensywność']],
inflection: [['M.', ['postęp'], ['postępy']], ['D.', ['postępu'], ['postępów']], ['C.', ['postępowi'], ['postępom']], ['B.', ['postęp'], ['postępy']], ['N.', ['postępem'], ['postępami']], ['Ms.', ['postępie'], ['postępach']], ['W.', ['postępie'], ['postępy']]]

word_id: 32253,
entry: metodologia,
part_of_speech: rzeczownik,
genders: ['ż'],
subentry: 1.a nauka,
categorization: [['CZŁOWIEK JAKO ISTOTA PSYCHICZNA', 'Działalność intelektualna człowieka', 'działalność naukowa']],
inflection: [['M.', ['metodologia'], ['metodologie']], ['D.', ['metodologii'], ['metodologii (neut)'], ['metodologij (char)']], ['C.', ['metodologii'], ['metodologiom']], ['B.', ['metodologię'], ['metodologie']], ['N.', ['metodologią'], ['metodologiami']], ['Ms.', ['metodologii'], ['metodologiach']], ['W.', ['metodologio'],

In [8]:
def find(entry):
    return [w for w in words if w.entry.lower() == entry]

def find_and_print(entry):
    for w in find(entry):
        print(w)

def find_any(entry):
    return find(entry)[0]

def find_and_print_any(entry):
     print(find_any(entry))

In [9]:
def get_case(word, case):
    for i in word.inflection:
        if len(i) > 1 and i[0] == case and len(i[1]) > 0:
            return i[1][0].split()[0]
    return None

def mianownik_lp(word):
    return get_case(word, "M.")

def dopelniacz_lp(word):
    return get_case(word, 'D.')
    
def celownik_lp(word):
    return get_case(word, "C.")

def biernik_lp(word):
    return get_case(word, "B.")

def narzednik_lp(word):
    return get_case(word, "N.")

def miejscownik_lp(word):
    return get_case(word, "Ms.")

def wolacz_lp(word):
    return get_case(word, "W.")

In [10]:
def is_present(case):
    return case is not None and len(case) > 0

def has_all_cases(word):
    if not is_present(mianownik_lp(word)):
        return False
    if not is_present(dopelniacz_lp(word)):
        return False
    if not is_present(celownik_lp(word)):
        return False
    if not is_present(biernik_lp(word)):
        return False
    if not is_present(narzednik_lp(word)):
        return False
    if not is_present(miejscownik_lp(word)):
        return False
    if not is_present(wolacz_lp(word)):
        return False
    return True

def is_standard(word):
    if not has_all_cases(word):
        return False
    
    mianownik = mianownik_lp(word)
    celownik = celownik_lp(word)
    
    if mianownik == celownik:
        return False    
    if len(mianownik) > 0 and mianownik[-1:] in ['a', 'i', 'y', 'o', 'ć']:
        return False
    if len(celownik) > 2 and celownik[-3:] == 'emu':
        return False
    return True 

def is_masculine(word):
    return len(set(word.genders) & {'m1', 'm2', 'm3'}) > 0

def is_masculine_animate(word):
    return len(set(word.genders) & {'m1', 'm2'}) > 0

def is_masculine_personal(word):
    return 'm1' in word.genders

def is_masculine_inanimate(word):
    return 'm3' in word.genders and len(set(word.genders) & {'m1', 'm2'}) == 0

def is_standard_masculine_noun(word):
    return is_standard(word) and is_masculine(word)

def is_standard_masculine_inanimate_noun(word):
    return is_standard(word) and is_masculine_inanimate(word)

def is_standard_masculine_animate_noun(word):
    return is_standard(word) and is_masculine_animate(word)

def is_standard_masculine_personal_noun(word):
    return is_standard(word) and is_masculine_personal(word)

In [11]:
masculine_nouns = [x for x in words if is_masculine(x)]
standard_masculine_nouns = [x for x in words if is_standard_masculine_noun(x)]

masculine_inanimate_nouns = [x for x in words if is_masculine_inanimate(x)]
standard_masculine_inanimate_nouns = [x for x in words if is_standard_masculine_inanimate_noun(x)]

masculine_animate_nouns = [x for x in words if is_masculine_animate(x)]
standard_masculine_animate_nouns = [x for x in words if is_standard_masculine_animate_noun(x)]

masculine_personal_nouns = [x for x in words if is_masculine_personal(x)]
standard_masculine_personal_nouns = [x for x in words if is_standard_masculine_personal_noun(x)]

print("masculine_nouns: ", len(masculine_nouns))
print("standard_masculine_nouns: ", len(standard_masculine_nouns))

print("masculine_inanimate_nouns: ", len(masculine_inanimate_nouns))
print("standard_masculine_inanimate_nouns: ", len(standard_masculine_inanimate_nouns))

print("masculine_animate_nouns: ", len(masculine_animate_nouns))
print("standard_masculine_animate_nouns: ", len(standard_masculine_animate_nouns))

print("masculine_personal_nouns: ", len(masculine_personal_nouns))
print("standard_masculine_personal_nouns: ", len(standard_masculine_personal_nouns))

masculine_nouns:  27028
standard_masculine_nouns:  24886
masculine_inanimate_nouns:  15972
standard_masculine_inanimate_nouns:  15640
masculine_animate_nouns:  11056
standard_masculine_animate_nouns:  9246
masculine_personal_nouns:  8133
standard_masculine_personal_nouns:  6417


In [12]:
def check_derivative(word1, word2):
    len1, len2 = len(word1), len(word2)

    if len1 < len2:
        return word2[-len1:] == word1
    else:
        return word1[-len2:] == word2


def print_words_with_derivatives(words):
    is_derivative = {word: False for word in words}
    derivatives = {word: [word] for word in words}

    for i in range(len(words)):
        for j in range(i + 1, len(words)):
            if check_derivative(words[i], words[j]):
                if len(words[i]) < len(words[j]):
                    is_derivative[words[j]] = True
                    derivatives[words[i]] += derivatives[words[j]]
                else:
                    is_derivative[words[i]] = True
                    derivatives[words[j]] += derivatives[words[i]]
    
    for word in words:
        if not is_derivative[word]:
            print(", ".join(derivatives[word]))

In [13]:
# Non-standard Celownik

incorrect_data = {
    'coolhunting',
    'goleń',
    'uporządkowanie'
}

nonstandard_celownik = set(x.entry.lower() for x in standard_masculine_nouns
                            if x.entry.lower() not in incorrect_data
                            and len(celownik_lp(x)) > 2
                            and celownik_lp(x)[-3:].lower() != 'owi')

print(len(nonstandard_celownik))
print_words_with_derivatives(list(nonstandard_celownik))

22
ksiądz, euroksiądz
bóg
lew
łeb
diabeł
książę, arcyksiążę
brat, psubrat
pies
ojciec, praojciec
kot, sukinkot
chłop, babochłop
pan, jaśniepan, waćpan
świat
chłopiec


In [14]:
# Non-standard Dopełniacz

incorrect_data = {}

nonstandard_dopelniacz = set(x.entry.lower() for x in standard_masculine_nouns
                          if x.entry.lower() not in incorrect_data
                          and dopelniacz_lp(x)[-1:] == 'a'
)
print(len(nonstandard_dopelniacz))

# Non-standard Biernik

incorrect_data = {}

nonstandard_biernik = set(x.entry.lower() for x in standard_masculine_nouns
                          if x.entry.lower() not in incorrect_data
                          and biernik_lp(x) == dopelniacz_lp(x)
                          and biernik_lp(x) != mianownik_lp(x)
)
print(len(nonstandard_biernik))

# TODO: categorize

8201
5909


In [15]:
vowels = ['a', 'ą', 'e', 'ę', 'i', 'o', 'ó', 'u', 'y',]
consonants = ['b', 'c', 'ć', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'ł', 'm', 'n', 'ń', 'p', 'q', 'r', 's', 'ś', 't', 'v', 'w', 'x', 'z', 'ź', 'ż']

def last_consonant(s):
    s_lower = s.lower()
    i = len(s_lower) - 1
    while i >= 0 and s_lower[i] not in consonants:
        i -= 1
    if i < 0:
        return None
    
    if s_lower[i] == 'z' and i > 0 and s_lower[i - 1] in ('c', 'r', 's', 'd', 't'):
        return s_lower[i - 1:i + 1]
    return s_lower[i]

def has_standard_masculine_miejscownik(word):
    mianownik = mianownik_lp(word).lower()
    miejscownik = miejscownik_lp(word).lower()

    if mianownik is None or miejscownik is None:
        return False
    
    last_cons = last_consonant(mianownik)
    if last_cons is None:
        return False
    
    if last_cons in ['b', 'w', 's', 'p', 'm', 'n', 'z', 'f', 'v']:
        return miejscownik[-3:] == last_cons + 'ie' \
                or miejscownik[-4:] == last_cons + '-ie' \
                or (last_cons == 'z' and miejscownik[-6:] == 'z-ecie')
    if last_cons == 'r':
        return miejscownik[-3:] == 'rze' or miejscownik[-4:] == 'r-ze'
    if last_cons == 'ł':
        return miejscownik[-2:] == 'le'
    if last_cons == 'd':
        return miejscownik[-4:] == 'dzie' or miejscownik[-5:] == 'd-zie'
    if last_cons == 't':
        return miejscownik[-3:] == 'cie' or miejscownik[-3:] == 'ć-u'
    if last_cons == 'ź':
        return miejscownik[-3:] == 'ziu' or miejscownik[-3:] == 'ź-u'
    if last_cons == 'ś':
        return miejscownik[-3:] == 'siu' or miejscownik[-3:] == 'ś-u'
    if last_cons == 'ć':
        return miejscownik[-3:] == 'ciu'
    if last_cons == 'ń':
        return miejscownik[-3:] == 'niu'
    if last_cons in ('cz', 'sz', 'rz', 'dz', 'tz'):
        return miejscownik[-3:] == last_cons + 'u' or miejscownik[-6:] == 'z-ecie'
    if last_cons == 'ż':
        return miejscownik[-2:] == 'żu' or miejscownik[-6:] == 'ż-ecie'
    if last_cons == 'x':
        return miejscownik[-4:] == 'ksie' or miejscownik[-4:] == 'x-ie'
    if last_cons == 'j':
        return miejscownik[-2:] == 'ju' or miejscownik[-3:] == 'j-u' or miejscownik[-6:] == 'j-ocie'
    if last_cons == 'q' and mianownik[-3:] == 'que':
        return miejscownik[-5:] == "que’u"
    if last_cons == 'c' and mianownik[-2:] == 'ce':
        return miejscownik[-3:] == "sie"
    if last_cons == 'c':
        return miejscownik[-2:] == "cu"  or miejscownik[-3:] == 'c-u' or miejscownik[-4:] == "c-ie"
    
    return miejscownik[-2:] == last_cons + 'u' or miejscownik[-3:] == last_cons + '-u' or miejscownik[-4:] == last_cons + "e’u" or miejscownik[-4:] == last_cons + "e'u"

In [16]:
# Non-standard Miejscownik

incorrect_data = {
    'UO',
    'uporządkowanie',
    'akap',
    'ankap',
    'szur i',
    'rai',
    'obsuw',
    'lockdown',
    'goleń',
    'hilal'
}

nonstandard_miejscownik1 = set(x.entry.lower() for x in standard_masculine_nouns
                            if x.entry.lower() not in incorrect_data
                            and miejscownik_lp(x).lower() != mianownik_lp(x).lower()
                            and not has_standard_masculine_miejscownik(x)
                            and miejscownik_lp(x)[-3:] in ('wiu', 'piu', 'biu', 'miu')
)

print(len(nonstandard_miejscownik1))
print_words_with_derivatives(list(nonstandard_miejscownik1))

28
radom
wab, jedwab
drób
żuraw
karp
gołąb
szczaw
kiełb
modrzew
oświęcim
wodzisław
wrocław, inowrocław
tułów
bytom
jastrząb
cietrzew
drop
czerw
badziew
ołów
ślep
paw
jarosław
nów
żółw
gap


In [17]:
nonstandard_miejscownik2 = set(x.entry.lower() for x in standard_masculine_nouns
                            if x.entry.lower() not in incorrect_data
                            and miejscownik_lp(x).lower() != mianownik_lp(x).lower()
                            and not has_standard_masculine_miejscownik(x)
                            and miejscownik_lp(x)[-3:] not in ('wiu', 'piu', 'biu', 'miu')
)

print(len(nonstandard_miejscownik2))
print_words_with_derivatives(list(nonstandard_miejscownik2))

8
pan, jaśniepan, waćpan
dom, ekodom
książę, arcyksiążę
syn


In [18]:
# Non-standard Wołacz

incorrect_data = {
    'wrn',
    'nfz',
    'szur i',
    'catcalling',
    'uporządkowanie',
    'późnorozwojowiec',
    'pewex',
    'lockdown',
    'sukinsyn',
    'skurczysyn',
    'skurwysyn',
    'ekodom',
    'pas',
    'obsuw',
    'pijar',
    'dziad',
    'akap',
    'ankap',
    'lud'
}

nonstandard_wolacz1 = set(x.entry.lower() for x in standard_masculine_nouns
                          if x.entry.lower() not in incorrect_data
                          and wolacz_lp(x) != miejscownik_lp(x)
                          and miejscownik_lp(x)[-2:] == 'cu'
                          and wolacz_lp(x)[-3:] == 'cze'
)

print(len(nonstandard_wolacz1))
print_words_with_derivatives(list(nonstandard_wolacz1))

14
pomazaniec
patrolowiec
krawiec
ojciec, praojciec
głupiec
zuchwalec
chłopiec
szewc
chciwiec
obrzydliwiec
ślepiec
młodzieniec
związkowiec


In [19]:
nonstandard_wolacz2 = set(x.entry.lower() for x in standard_masculine_nouns
                          if x.entry.lower() not in incorrect_data
                          and wolacz_lp(x) != miejscownik_lp(x)
                          and miejscownik_lp(x)[-2:] != 'cu'
)

print(len(nonstandard_wolacz2))
print_words_with_derivatives(list(nonstandard_wolacz2))

set(f"{x.entry.lower()}:{miejscownik_lp(x)}:{wolacz_lp(x)}" for x in words if x.entry.lower() in nonstandard_wolacz2)


12
podczłowiek
ksiądz, euroksiądz
białystok
pan, jaśniepan, waćpan
bóg
człek
krasnystaw
książę, arcyksiążę


{'arcyksiążę:arcyksięciu:arcyksiążę',
 'białystok:Białymstoku:Białystoku',
 'bóg:Bogu:Boże',
 'bóg:bogu:boże',
 'człek:człeku:człecze',
 'euroksiądz:euroksiędzu:euroksięże',
 'jaśniepan:jaśniepanu:jaśniepanie',
 'krasnystaw:Krasnymstawie:Krasnystawie',
 'ksiądz:księdzu:księże',
 'książę:księciu:książę',
 'pan:PAN:PAN',
 'pan:Panu:Panie',
 'pan:panu:panie',
 'podczłowiek:podczłowieku:podczłowiecze',
 'waćpan:waćpanu:waćpanie'}

In [20]:
set([x.entry.lower() for x in masculine_personal_nouns if len(x.entry.lower()) >= 3 and x.entry.lower()[-3:] in ('ist', 'yst')])

{'antychryst'}